<h1 style="color: white; background: #ADD8E6; text-align:center; font-size: 2.3em;"> MoA + Feature Engeneering + Keras </h1>

## File Paths

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/iterative-stratification/iterative-stratification-master/.gitignore
/kaggle/input/iterative-stratification/iterative-stratification-master/LICENSE
/kaggle/input/iterative-stratification/iterative-stratification-master/README.md
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.cfg
/kaggle/input/iterative-stratification/iterative-stratification-master/setup.py
/kaggle/input/iterative-stratification/iterative-stratification-master/.travis.yml
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/__init__.py
/kaggle/input/iterative-stratification/iterative-stratification-master/tests/test_ml_stratifiers.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/__init__.py
/kaggle/input/iterative-stratification/iterative-stratification-master/iterstrat/ml_stratifiers.py
/kaggle/input/rank-gauss/rankGaussTrafo.py
/kaggle/input/rank-gauss/gauss_rank_scaler.py
/kaggle/input/rank-gauss/rgn.py
/kaggle/inpu

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.callbacks as C
import tensorflow.keras.layers as L
import tensorflow.keras.models as M
import tensorflow.keras.optimizers as O
import random
import os
from sklearn.preprocessing import StandardScaler, MinMaxScaler, QuantileTransformer, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from sklearn.decomposition import PCA
from scipy import stats
import sys
sys.path.append('../input/iterative-stratification/iterative-stratification-master')
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
sys.path.append('../input/rank-gauss')
from gauss_rank_scaler import GaussRankScaler
import json

/opt/conda/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:68: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.3.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


<h3 style="color: 	#32CD32;">Reading Files</h3>

In [3]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
train_non = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
sub = pd.read_csv('../input/lish-moa/sample_submission.csv')

In [4]:
train_features2 = pd.read_csv('../input/lish-moa/train_features.csv')
test_features2 = pd.read_csv('../input/lish-moa/test_features.csv')

In [5]:
# json_file_path = '../input/t-test-pca-rfe-logistic-regression/main_predictors.json'

# with open(json_file_path, 'r') as j:
#     predictors = json.loads(j.read())
#     predictors = predictors['start_predictors']

In [6]:
# predictors = []
# for col in train_features.columns:
#     if col in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']:
#         continue
#     if stats.ttest_ind(train_features[col], test_features[col]).pvalue > 0.8:
#         predictors.append(col)

In [7]:
predictors = ['g-13','g-17','g-20','g-29','g-30','g-33','g-42','g-53','g-54','g-73','g-83','g-90','g-103','g-115','g-122','g-129','g-130','g-132','g-141','g-157','g-164','g-176','g-196','g-199','g-200','g-202','g-207','g-219','g-226','g-227','g-247','g-259','g-260','g-268','g-270','g-278','g-288','g-292','g-303','g-308','g-313','g-317','g-319','g-328','g-333','g-337','g-348','g-352','g-366','g-368','g-372','g-382','g-388','g-394','g-417','g-424','g-425','g-434','g-435','g-436','g-442','g-453','g-462','g-463','g-471','g-476','g-483','g-484','g-512','g-520','g-524','g-527','g-530','g-538','g-544','g-549','g-552','g-566','g-570','g-581','g-583','g-592','g-594','g-597','g-601','g-611','g-621','g-626','g-636','g-641','g-643','g-644','g-646','g-649','g-653','g-658','g-662','g-671','g-701','g-711','g-713','g-714','g-715','g-722','g-729','g-737','c-0','c-16','c-32','c-53','c-89']

In [8]:
target_cols = sub.columns[1:]

In [9]:
len(predictors)

111

### Set Seed for Reproducibility

In [10]:
def seed_everything(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

### Feature Engineering/Selection

In [11]:
#All the col that has genes and cells
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [12]:
# We are actully adding the feature from the pca into the original data in here, it helps the score for some reason
def pca(train, test, n_comp = 55, type_ = GENES, SEED = 123):
    
    kind = "g" if type_ == GENES else "c"
    data = pd.concat([train[type_], test[type_]])
    pca = PCA(n_components= n_comp, random_state = SEED)
    data = pd.DataFrame(pca.fit_transform(data), columns = [f'pca_{kind}{i + 1}' for i in range(n_comp)])
    train_ = data.iloc[:train.shape[0]]
    test_ = data.iloc[train.shape[0]:].reset_index(drop = True)
    train = pd.concat([train, train_], axis = 1)
    test = pd.concat([test, test_], axis = 1)
    return train, test

In [13]:
train_features = train_features[train_features["cp_type"] != "ctl_vehicle"]
test_features = test_features[test_features["cp_type"] != "ctl_vehicle"]
test_features2 = test_features[test_features["cp_type"] != "ctl_vehicle"]
train_targets = train_targets.iloc[train_features.index]
train_features.reset_index(drop = True, inplace = True)
test_features.reset_index(drop = True, inplace = True)
test_features2.reset_index(drop = True, inplace = True)
train_targets.reset_index(drop = True, inplace = True)

In [14]:
train_features, test_features = pca(train_features, test_features, n_comp = 80, type_ = GENES)
train_features, test_features = pca(train_features, test_features, n_comp = 15, type_ = CELLS)

In [15]:
#Feature Selection with Variance Threshold
#Had problems with submission on this
c_n = [f for f in list(train_features.columns) if f not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]
mask = (train_features[c_n].var() >= 0.65).values
tmp = train_features[c_n].loc[:, mask]
train_features = pd.concat([train_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
tmp = test_features[c_n].loc[:, mask]
test_features = pd.concat([test_features[['sig_id', 'cp_type', 'cp_time', 'cp_dose']], tmp], axis=1)
cols_numeric = [feat for feat in list(train_features.columns) if feat not in ['sig_id', 'cp_type', 'cp_time', 'cp_dose']]

In [16]:
#All the col that has genes and cells
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [17]:
def create_cluster(train, test, kind, n_cluster, str_kind):
    
    data = pd.concat([train[kind], test[kind]], ignore_index = True)
    kmeans = KMeans(n_clusters = n_cluster, random_state = 0,).fit(data)
    train[f'clusters_{str_kind}'] = kmeans.labels_[:train.shape[0]]
    test[f'clusters_{str_kind}'] = kmeans.labels_[train.shape[0]:]
    train = pd.get_dummies(train, columns = [f'clusters_{str_kind}'])
    test = pd.get_dummies(test, columns = [f'clusters_{str_kind}'])
    
    return train, test

In [18]:
# train_features, test_features = create_cluster(train_features, test_features, GENES, 35, "g")
# train_features, test_features = create_cluster(train_features, test_features, CELLS, 5, "c")

In [19]:
data = pd.concat([train_features, test_features], ignore_index = True)

In [20]:
#Normalize the data
scaler = GaussRankScaler()
data[cols_numeric]  = scaler.fit_transform(data[cols_numeric])

In [21]:
#Spliting the data back into train and test
train_features = data.iloc[:train_features.shape[0]].reset_index(drop = True)
test_features = data.iloc[train_features.shape[0]:].reset_index(drop = True)

In [22]:
def fe_stats(train, test):
    
    features_g = list(train.columns[4:776])
    features_c = list(train.columns[776:876])
    
    for df in [train, test]:
        df['g_sum'] = df[features_g].sum(axis = 1)
        df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1)
        df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        df['g_skew'] = df[features_g].skew(axis = 1)
        df['c_sum'] = df[features_c].sum(axis = 1)
        df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1)
        df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        df['c_skew'] = df[features_c].skew(axis = 1)
        df['gc_sum'] = df[features_g + features_c].sum(axis = 1)
        df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        df['gc_std'] = df[features_g + features_c].std(axis = 1)
        df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        
    return train, test

## Preprocessing

In [23]:
def preprocess(data, target=False):
    data.drop("sig_id", axis=1, inplace=True)
    if target:
        return data
    else:
        data["cp_dose_time"] = data["cp_dose"] + "_" + data["cp_time"].astype("str")
        data["cp_dose_time"] = data["cp_dose_time"].map({"D1_24":0 , "D1_48":1, "D1_72":2, "D2_24":3 , "D2_48":4, "D2_72":5})
        data["cp_time"] = data["cp_time"].map({24:1, 48:2, 72:3})
        #Doing this slightly improves the model
        #https://www.kaggle.com/carlmcbrideellis/moa-setting-ctl-vehicle-0-improves-score
        data.at[data['cp_type'].str.contains('ctl_vehicle'),data.filter(regex='-.*').columns] = 0.0
    return data

In [24]:
train_features = preprocess(train_features)
train_targets = preprocess(train_targets, target=True)
train_features = pd.get_dummies(train_features)
test_features = preprocess(test_features)
test_features = pd.get_dummies(test_features)

In [25]:
train_features, test_features = fe_stats(train_features, test_features)

In [26]:
train_features

,cp_time,g-0,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,c_sum,c_mean,c_std,c_kurt,c_skew,gc_sum,gc_mean,gc_std,gc_kurt,gc_skew
0,1,0.731433,-0.254104,-0.615315,-0.194236,-0.772257,-1.061067,0.005623,0.513989,-0.130963,...,29.950827,0.299508,0.552298,0.115705,-0.192222,39.268721,0.045033,0.604015,-0.065454,-0.036394
1,3,0.017020,0.213820,0.027107,0.849834,0.482972,0.217779,0.363904,-0.298420,0.730029,...,29.694569,0.296946,0.477884,0.002827,-0.400339,42.978577,0.049287,0.584949,-0.125831,0.005071
2,2,0.462792,0.981712,-0.091777,-0.039121,1.055526,0.161005,0.242395,0.050070,0.975415,...,-4.183705,-0.041837,0.507697,-0.395305,0.193820,-19.827301,-0.022738,0.622199,-0.309186,-0.002089
3,2,-0.531916,-0.269127,0.455184,1.595325,-0.621171,-1.509230,0.206249,-0.066407,-0.889590,...,-54.176846,-0.541768,0.585042,1.558884,1.458391,-79.440660,-0.091102,0.733257,-0.931805,0.258173
4,3,-0.341668,0.808567,0.604615,0.951946,-0.630950,-0.243933,-0.103294,-0.684346,0.741863,...,22.473358,0.224734,0.565969,-0.116749,-0.319863,22.767152,0.026109,0.743760,-0.278707,-0.196499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,3,0.089366,0.176202,-0.226762,-0.244678,0.396468,-0.094758,-0.130161,-0.582338,0.056587,...,8.616747,0.086167,0.503629,-0.177283,0.462609,48.865715,0.056039,0.479665,-0.048608,-0.005369
21944,1,0.072056,-0.131998,-0.499092,-0.479222,0.667644,0.530397,0.333563,-0.356318,0.048109,...,23.792189,0.237922,0.509802,-0.278833,0.097052,37.947722,0.043518,0.544859,0.228663,-0.167903
21945,1,-1.346695,-0.369847,0.878667,-0.735139,0.613056,-0.194656,-0.446960,0.479298,0.549999,...,3.316741,0.033167,0.562406,-0.320131,0.404143,8.818301,0.010113,0.554599,0.104428,0.089995
21946,1,0.490027,0.329593,0.157679,0.732402,0.001062,0.034881,0.079245,0.405150,-0.742506,...,31.153129,0.311531,0.738239,-0.326836,-0.320763,87.500526,0.100345,0.780387,-0.947005,-0.175951


### Error/Loss Metric and Prediction clipping

In [27]:
def multi_log_loss(y_true, y_pred):
    losses = []
    for col in y_true.columns:
        losses.append(log_loss(y_true.loc[:, col], y_pred.loc[:, col]))
    return np.mean(losses)

In [28]:
def logloss(y_true, y_pred):
    y_pred = tf.clip_by_value(y_pred,p_min,p_max)
    return -tf.keras.backend.mean(y_true*tf.keras.backend.log(y_pred) + (1-y_true)*tf.keras.backend.log(1-y_pred))

In [29]:
def clip(y_pred, c_min, c_max):
    return np.clip(y_pred, c_min, c_max)

In [30]:
p_min = 0.001
p_max = 0.999

## Create Model and Perform KFold

In [31]:
def resnet(n_features, n_features_2, n_labels, label_smoothing = 0.0005):    
    input_1 = L.Input(shape = (n_features,), name = 'Input1')
    input_2 = L.Input(shape = (n_features_2,), name = 'Input2')

    head_1 = M.Sequential([
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(512), 
        L.LeakyReLU(alpha = 0.10059420295821832),
        L.BatchNormalization(),
        L.Dense(256),
        L.Dropout(0.2),
        L.LeakyReLU(alpha = 0.10059420295821832),
        L.Dense(256),
        L.LeakyReLU(alpha = 0.10059420295821832),
        ],name='Head1') 

    input_3 = head_1(input_1)
    input_3_concat = L.Concatenate()([input_2, input_3])

    head_2 = M.Sequential([
        L.BatchNormalization(),
        L.Dropout(0.3),
        L.Dense(1024),
        L.LeakyReLU(alpha = 0.10059420295821832),
        L.BatchNormalization(),
        L.Dropout(0.4),
        L.Dense(512, "elu"),
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(256),
        L.LeakyReLU(alpha = 0.10059420295821832),
        L.BatchNormalization(),
        L.Dense(256, "elu"),
        ],name='Head2')

    input_4 = head_2(input_3_concat)
    input_4_avg = L.Average()([input_3, input_4]) 

    head_3 = M.Sequential([
        L.BatchNormalization(),
        L.Dense(512, kernel_initializer='lecun_normal', activation='selu'),
        L.BatchNormalization(),
        L.Dropout(0.2),
        L.Dense(n_labels, kernel_initializer='lecun_normal', activation='selu'),
        L.BatchNormalization(),
        L.Dense(n_labels, activation="sigmoid")
        ],name='Head3')

    output = head_3(input_4_avg)


    model = M.Model(inputs = [input_1, input_2], outputs = output)
    model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing), metrics=logloss)
    
    return model


In [32]:
def make_folds(folds):
    models = []
    oof_preds_score = []
    for i in range(5):
        
        seed = 34 + i
        seed_everything(seed)
        print(f"REAPEAT NUMBER {i + 1} SEED {seed}")
        
        model_list = list()
        kfold = MultilabelStratifiedKFold(folds, shuffle = True)
        oof_preds = train_targets.copy()
        
        
        for fold, (train_ind, val_ind) in enumerate(kfold.split(train_targets, train_targets)):
            
            print(f"fold {fold + 1}")
            print("\n")
            print("ResNet")
            
            train2 = train_features2.iloc[train_ind][predictors].values
            train_val2 = train_features2.iloc[val_ind][predictors].values
            
            model = resnet(train_features.shape[1], train2.shape[1], 206)
            
            checkpoint_path = f'repeat:{seed}_Fold:{fold}.hdf5'
            cb_checkpt = C.ModelCheckpoint(checkpoint_path, monitor = 'val_logloss', verbose = 1, save_best_only = True, save_weights_only = True, mode = 'auto')
            reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = 'val_logloss',
                                                         mode = 'min',
                                                         factor = 0.1,
                                                         patience = 2,
                                                         verbose = 1)
            early = tf.keras.callbacks.EarlyStopping(monitor = 'val_logloss',
                                                          mode = 'min',
                                                          patience = 6,
                                                          restore_best_weights = True,
                                                          verbose = 1)
            
            model.fit([train_features.values[train_ind], train2],
                    train_targets.values[train_ind], validation_data=([train_features.values[val_ind], train_val2], train_targets.values[val_ind]),
                    epochs=100, batch_size=128, verbose=2, callbacks = [reduce_lr, early, cb_checkpt])
            model_list.append(model)
            model.load_weights(checkpoint_path)
            oof_preds.loc[val_ind, :] = clip(model.predict([train_features.values[val_ind], train_val2]), p_min, p_max)
            
        m, oof = model_list, oof_preds
        models = models + m
        oof_preds_score.append(oof)
    return models, oof_preds_score

In [33]:
models, oof_preds = make_folds(10)

REAPEAT NUMBER 1 SEED 34


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fold 1


ResNet
Epoch 1/100

Epoch 00001: val_logloss improved from inf to 0.04146, saving model to repeat:34_Fold:0.hdf5
155/155 - 2s - loss: 0.3439 - logloss: 0.3422 - val_loss: 0.0424 - val_logloss: 0.0415
Epoch 2/100

Epoch 00002: val_logloss improved from 0.04146 to 0.02215, saving model to repeat:34_Fold:0.hdf5
155/155 - 1s - loss: 0.0284 - logloss: 0.0273 - val_loss: 0.0235 - val_logloss: 0.0222
Epoch 3/100

Epoch 00003: val_logloss improved from 0.02215 to 0.01886, saving model to repeat:34_Fold:0.hdf5
155/155 - 1s - loss: 0.0217 - logloss: 0.0203 - val_loss: 0.0203 - val_logloss: 0.0189
Epoch 4/100

Epoch 00004: val_logloss improved from 0.01886 to 0.01786, saving model to repeat:34_Fold:0.hdf5
155/155 - 1s - loss: 0.0199 - logloss: 0.0185 - val_loss: 0.0194 - val_logloss: 0.0179
Epoch 5/100

Epoch 00005: val_logloss improved from 0.01786 to 0.01754, saving model to repeat:34_Fold:0.hdf5
155/155 - 2s - loss: 0.0191 - logloss: 0.0177 - val_loss: 0.0191 - val_logloss: 0.0175
Epo

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fold 1


ResNet
Epoch 1/100

Epoch 00001: val_logloss improved from inf to 0.04557, saving model to repeat:35_Fold:0.hdf5
155/155 - 2s - loss: 0.3463 - logloss: 0.3446 - val_loss: 0.0466 - val_logloss: 0.0456
Epoch 2/100

Epoch 00002: val_logloss improved from 0.04557 to 0.02129, saving model to repeat:35_Fold:0.hdf5
155/155 - 1s - loss: 0.0289 - logloss: 0.0278 - val_loss: 0.0228 - val_logloss: 0.0213
Epoch 3/100

Epoch 00003: val_logloss improved from 0.02129 to 0.01881, saving model to repeat:35_Fold:0.hdf5
155/155 - 1s - loss: 0.0219 - logloss: 0.0206 - val_loss: 0.0205 - val_logloss: 0.0188
Epoch 4/100

Epoch 00004: val_logloss improved from 0.01881 to 0.01788, saving model to repeat:35_Fold:0.hdf5
155/155 - 1s - loss: 0.0201 - logloss: 0.0187 - val_loss: 0.0196 - val_logloss: 0.0179
Epoch 5/100

Epoch 00005: val_logloss improved from 0.01788 to 0.01760, saving model to repeat:35_Fold:0.hdf5
155/155 - 1s - loss: 0.0193 - logloss: 0.0179 - val_loss: 0.0193 - val_logloss: 0.0176
Epo

...

fold 1


ResNet
Epoch 1/100

Epoch 00001: val_logloss improved from inf to 0.04218, saving model to repeat:36_Fold:0.hdf5
155/155 - 2s - loss: 0.3442 - logloss: 0.3425 - val_loss: 0.0432 - val_logloss: 0.0422
Epoch 2/100

Epoch 00002: val_logloss improved from 0.04218 to 0.02176, saving model to repeat:36_Fold:0.hdf5
155/155 - 1s - loss: 0.0284 - logloss: 0.0274 - val_loss: 0.0231 - val_logloss: 0.0218
Epoch 3/100

Epoch 00003: val_logloss improved from 0.02176 to 0.01907, saving model to repeat:36_Fold:0.hdf5
155/155 - 1s - loss: 0.0216 - logloss: 0.0203 - val_loss: 0.0205 - val_logloss: 0.0191
Epoch 4/100

Epoch 00004: val_logloss improved from 0.01907 to 0.01803, saving model to repeat:36_Fold:0.hdf5
155/155 - 2s - loss: 0.0200 - logloss: 0.0186 - val_loss: 0.0196 - val_logloss: 0.0180
Epoch 5/100

Epoch 00005: val_logloss improved from 0.01803 to 0.01763, saving model to repeat:36_Fold:0.hdf5
155/155 - 2s - loss: 0.0192 - logloss: 0.0177 - val_loss: 0.0192 - val_logloss: 0.0176
Epo

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fold 1


ResNet
Epoch 1/100

Epoch 00001: val_logloss improved from inf to 0.04159, saving model to repeat:37_Fold:0.hdf5
155/155 - 2s - loss: 0.3438 - logloss: 0.3421 - val_loss: 0.0426 - val_logloss: 0.0416
Epoch 2/100

Epoch 00002: val_logloss improved from 0.04159 to 0.02142, saving model to repeat:37_Fold:0.hdf5
155/155 - 2s - loss: 0.0285 - logloss: 0.0274 - val_loss: 0.0228 - val_logloss: 0.0214
Epoch 3/100

Epoch 00003: val_logloss improved from 0.02142 to 0.01892, saving model to repeat:37_Fold:0.hdf5
155/155 - 2s - loss: 0.0215 - logloss: 0.0202 - val_loss: 0.0204 - val_logloss: 0.0189
Epoch 4/100

Epoch 00004: val_logloss improved from 0.01892 to 0.01781, saving model to repeat:37_Fold:0.hdf5
155/155 - 1s - loss: 0.0199 - logloss: 0.0185 - val_loss: 0.0194 - val_logloss: 0.0178
Epoch 5/100

Epoch 00005: val_logloss improved from 0.01781 to 0.01742, saving model to repeat:37_Fold:0.hdf5
155/155 - 2s - loss: 0.0191 - logloss: 0.0176 - val_loss: 0.0190 - val_logloss: 0.0174
Epo

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass shuffle=True, random_state=None as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


fold 1


ResNet
Epoch 1/100

Epoch 00001: val_logloss improved from inf to 0.04161, saving model to repeat:38_Fold:0.hdf5
155/155 - 2s - loss: 0.3424 - logloss: 0.3407 - val_loss: 0.0426 - val_logloss: 0.0416
Epoch 2/100

Epoch 00002: val_logloss improved from 0.04161 to 0.02237, saving model to repeat:38_Fold:0.hdf5
155/155 - 2s - loss: 0.0287 - logloss: 0.0276 - val_loss: 0.0238 - val_logloss: 0.0224
Epoch 3/100

Epoch 00003: val_logloss improved from 0.02237 to 0.01927, saving model to repeat:38_Fold:0.hdf5
155/155 - 2s - loss: 0.0218 - logloss: 0.0205 - val_loss: 0.0209 - val_logloss: 0.0193
Epoch 4/100

Epoch 00004: val_logloss improved from 0.01927 to 0.01799, saving model to repeat:38_Fold:0.hdf5
155/155 - 2s - loss: 0.0201 - logloss: 0.0187 - val_loss: 0.0198 - val_logloss: 0.0180
Epoch 5/100

Epoch 00005: val_logloss improved from 0.01799 to 0.01733, saving model to repeat:38_Fold:0.hdf5
155/155 - 2s - loss: 0.0192 - logloss: 0.0177 - val_loss: 0.0192 - val_logloss: 0.0173
Epo

In [34]:
mean_oof_preds = train_targets.copy()
mean_oof_preds.loc[:, target_cols] = 0
scores = []
for i, p in enumerate(oof_preds):
    loss = multi_log_loss(train_targets, p)
    print("LEAKY RELU SCORE")
    print(f"Repeat {i + 1} OOF Log Loss: {loss}")
    scores.append(loss)
    mean_oof_preds.loc[:,target_cols] += p
mean_oof_preds.loc[:, target_cols] /= len(oof_preds)
print(f"Mean OOF Log Loss: {multi_log_loss(train_targets, mean_oof_preds)}")

LEAKY RELU SCORE
Repeat 1 OOF Log Loss: 0.016543303267814467
LEAKY RELU SCORE
Repeat 2 OOF Log Loss: 0.016576982988045965
LEAKY RELU SCORE
Repeat 3 OOF Log Loss: 0.016511309522550553
LEAKY RELU SCORE
Repeat 4 OOF Log Loss: 0.01655908453466226
LEAKY RELU SCORE
Repeat 5 OOF Log Loss: 0.016534854065383058
Mean OOF Log Loss: 0.0160795758831711


## Submission File

In [35]:
test_ = pd.read_csv('../input/lish-moa/test_features.csv')

In [36]:
sig_id = test_[test_["cp_type"] != "ctl_vehicle"].sig_id.reset_index(drop = True)

In [37]:
test_preds = sub.copy()
test_preds[target_cols] = 0
test_preds = test_preds.drop("sig_id", axis = 1)
test_preds["sig_id"] = sig_id
test_preds.dropna(inplace=True)
test_preds.drop("sig_id", axis = 1, inplace=True)
for model in models:
    preds = model.predict([test_features.values, test_features2[predictors].values])
    test_preds.loc[:,target_cols] += clip(preds, p_min, p_max)
test_preds.loc[:,target_cols] /= len(models)
test_preds["sig_id"] = sig_id
test_features = pd.read_csv('../input/lish-moa/test_features.csv')
submission = pd.merge(test_features[["sig_id"]], test_preds, on = "sig_id", how = "left")
submission.fillna(0, inplace = True)
sub = submission.to_csv('submission.csv', index=False)

In [38]:
test_features

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99
0,id_0004d9e33,trt_cp,24,D1,-0.5458,0.1306,-0.5135,0.4408,1.5500,-0.1644,...,0.0981,0.7978,-0.1430,-0.2067,-0.2303,-0.1193,0.0210,-0.0502,0.1510,-0.7750
1,id_001897cda,trt_cp,72,D1,-0.1829,0.2320,1.2080,-0.4522,-0.3652,-0.3319,...,-0.1190,-0.1852,-1.0310,-1.3670,-0.3690,-0.5382,0.0359,-0.4764,-1.3810,-0.7300
2,id_002429b5b,ctl_vehicle,24,D1,0.1852,-0.1404,-0.3911,0.1310,-1.4380,0.2455,...,-0.2261,0.3370,-1.3840,0.8604,-1.9530,-1.0140,0.8662,1.0160,0.4924,-0.1942
3,id_00276f245,trt_cp,24,D2,0.4828,0.1955,0.3825,0.4244,-0.5855,-1.2020,...,0.1260,0.1570,-0.1784,-1.1200,-0.4325,-0.9005,0.8131,-0.1305,0.5645,-0.5809
4,id_0027f1083,trt_cp,48,D1,-0.3979,-1.2680,1.9130,0.2057,-0.5864,-0.0166,...,0.4965,0.7578,-0.1580,1.0510,0.5742,1.0900,-0.2962,-0.5313,0.9931,1.8380
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3977,id_ff7004b87,trt_cp,24,D1,0.4571,-0.5743,3.3930,-0.6202,0.8557,1.6240,...,-1.1790,-0.6422,-0.4367,0.0159,-0.6539,-0.4791,-1.2680,-1.1280,-0.4167,-0.6600
3978,id_ff925dd0d,trt_cp,24,D1,-0.5885,-0.2548,2.5850,0.3456,0.4401,0.3107,...,0.0210,0.5780,-0.5888,0.8057,0.9312,1.2730,0.2614,-0.2790,-0.0131,-0.0934
3979,id_ffb710450,trt_cp,72,D1,-0.3985,-0.1554,0.2677,-0.6813,0.0152,0.4791,...,0.4418,0.9153,-0.1862,0.4049,0.9568,0.4666,0.0461,0.5888,-0.4205,-0.1504
3980,id_ffbb869f2,trt_cp,48,D2,-1.0960,-1.7750,-0.3977,1.0160,-1.3350,-0.2207,...,0.3079,-0.4473,-0.8192,0.7785,0.3133,0.1286,-0.2618,0.5074,0.7430,-0.0484


In [39]:
def Diff(list1, list2): 
    return (list(list(set(list1)-set(list2)) + list(set(list2)-set(list1)))) 

Diff (test_preds.columns, pd.read_csv('../input/lish-moa/sample_submission.csv').columns)

[]

In [40]:
len(Diff(train_targets, train_non))

609